# Project Code Draft ##

## Import Statements ##

In [2]:
import scipy.io as sio
import inspect
import os
import contextlib
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
sns.set()

## Data Pre-processing Functions ##

In [3]:
#function-type alias for use in type hints
function = type(lambda x: x)

######################## Data Dimensionality Reduction Methods ##############################################################
def sliding_window(X:np.ndarray, y:np.ndarray, window_length:int, step_size:int, step:int):
    w_start_idx = step_size*step
    w_end_idx = min( [X.shape[1]-1, w_start_idx + window_length] )
    if y is not None:
        return X[:, w_start_idx:w_end_idx, :], y[w_start_idx:w_end_idx]
    if y is None:
        return X[:, w_start_idx:w_end_idx, :]

#################### Laterized Readiness Potential Logic
def calc_lrp(X:np.ndarray): 
    X_reduced = X[23,:,:] - X[7,:,:]
    return X_reduced.T

def sliding_lrp(X:np.ndarray, y:np.ndarray, window_length:int, step_size:int, step:int):
    X_window, y_window = sliding_window(X, window_length, step_size, step)
    return calc_lrp(X_window), y_window



######################## MNE CSP transform logic

#Constants to save last-trained CSP objects
# and dictionary of saved CSP objects for later use
csp_transformer = mne.decoding.CSP()
csp_dict = {}

def csp_transform(X:np.ndarray, y:np.ndarray=None, n_components:int=None, transformer_label:str=None):
    global csp_transformer
    global csp_dict
    dim = X.shape
    X_shaped = X.reshape(dim[2], dim[0], dim[1])
    if y is not None:
        if n_components is not None:
            csp_transformer = mne.decoding.CSP(n_components)
        else:
            csp_transformer = mne.decoding.CSP()
        #Supress lengthy print statements from CSP
                    #WARNING: This will supress all warnings and errors. ONLY USE IF CODE HAS BEEN TESTED
                    # AND IT IS KNOWN THAT IT WILL NOT CRASH DURING THE FITTING PROCEDURE 
        with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
            csp_transformer.fit(X_shaped, y)
        if transformer_label is not None:
            csp_dict[transformer_label] = csp_transformer
        return None        
    else:
        #Supress lengthy print statements from CSP
                    #WARNING: This will supress all warnings and errors. ONLY USE IF CODE HAS BEEN TESTED
                    # AND IT IS KNOWN THAT IT WILL NOT CRASH DURING THE FITTING PROCEDURE 
        with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
            if transformer_label is not None:
                return csp_dict[transformer_label].transform(X_shaped)
            else:
                return csp_transformer.transform(X_shaped)

def sliding_csp_transform(X:np.ndarray,
                          window_length:int, 
                          step_size:int,
                          step:int,
                          y:np.ndarray=None,
                          n_components:int=None,
                          transformer_label:str=None,
                          refit_csp=False):
    X_window, y_window = sliding_window(X, window_length, step_size, step)
    if refit_csp:
        csp_transform(X_window, y_window, n_components, transformer_label)
    return csp_transform(X_window, transformer_label=transformer_label), y_window
    
def sliding_csp_fit_transform(X:np.ndarray,
                              window_length:int,
                              step_size:int,
                              step:int,
                              y:np.ndarray,
                              n_components:int=None,
                              transformer_label:str=None):
    X_window, y_window = sliding_window(X, window_length, step_size, step)
    X_tr, X_ts, y_tr, y_ts = train_test_split(X_window, y_window, train_size=0.3)
    csp_transform(X_tr, y_tr, n_components, transfomer_label)
    return csp_transform(X_ts), y_ts, X_tr, y_tr
    
######################################### Data Formatting Logic ################################################
def Format(X:np.ndarray, 
           y:np.ndarray,  
           reduction_method:function, 
           reduction_method_args=None,
           trials_in_A:int=100):
    if type(reduction_method_args) == dict:
        X_reduced = reduction_method(X, **reduction_method_args)
    elif type(reduction_method_args) == list:
        X_reduced = reduction_method(X, *reduction_method_args)
    else:
        X_reduced = reduction_method(X)
    condition_A_data_dict, condition_B_data_dict = condition_split(X_reduced, y, trials_in_A)
    return condition_A_data_dict, condition_B_data_dict

def condition_split(X:np.ndarray, y:np.ndarray, trials_in_A:int):
    AX, BX = X[:trials_in_A], X[trials_in_A:]
    Ay, By = y[:trials_in_A], y[trials_in_A:]
    A_dict = {'X': AX, 'y': Ay}
    B_dict = {'X': BX, 'y': By}
    return A_dict, B_dict


## Hyperparameter Tuning ##

#### Read in Hyperparameter Tuning Data ####

In [4]:
data = sio.loadmat('../Data/data_cube_subject1.mat')
channel_labels = sio.loadmat('../Data/channel_label.mat')
data_cube = data['data_cube']
data_labels = data['event_label'].ravel()

In [5]:
#Divide data into CSP tuning and Hyperparameter Tuning Data
data_cube.shape
X_csp = data_cube[:,:,80:120]
y_csp = data_labels[80:120]
X_param_csp = np.vstack([data_cube[:,:,:80], data_cube[:,:,120:]])
y_param_csp = np.hstack([data_labels[:80],data_labels[120:]])

#fit CSP to proper data and save CSP object
csp_transform(X_csp, y_csp, transformer_label='hyperparam_tune')

#### Hyperparameter Tuning Constants ####

In [6]:
#Hyperparameter dictionaries for use in grid search
ada_params = {
    'n_estimators': [50, 100, 200, 500],
    'learning_rate': [x/20 for x in range(1,21)],
    'algorithm': ['SAMME', 'SAMME.R']
}

rf_params = {
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': (None, 10, 50, 100),
    'min_samples_split': [2, 4, 5],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ('auto', 'sqrt', 'log2', None),
    'min_impurity_decrease': (0.0, 0.5)
}

lda_params = {
    'solver': ['svd','lsqr','eigen'],
    'shrinkage': ['auto',None],
}

svm_params = {
    'C': np.logspace(-3,2,6),
    'kernel': ['linear','poly','rbf','sigmoid'],
    'degree': [2,3,4],
    'gamma': ['scale', 'auto'] + list(np.logspace(-3,3,6)),
    'shrinking': [True, False],
    'max_iter': [1000]
}

#Dictionary of classifiers and hyperparemeter selections
clfs = {
    'LDA': LDA(),
    'SVM': SVC(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
}

params = {
    'RandomForestClassifier': rf_params,
    'AdaBoostClassifier': ada_params,
    'LDA': lda_params,
    'SVM': svm_params
}

#data dimensionality reduction methods to opitmize for
    #note that csp transform will use n_components=4 for model hyperparameter tuning
    # n_components will be finalized in another round of cross-validation
    # runtime to validate all combinations of n_components and model hyperparameters too long
    # for our current computational capacity so this will have to do
reduction_methods = {
    'lrp': calc_lrp,
    'csp': lambda X: csp_transform(X, transformer_label='hyperparam_tune')
}

#### Hyperparameter Tuning Functions ####

In [7]:

def tune_multiple_models(data_matrix:np.ndarray, 
                         data_labels:np.ndarray, 
                         clf_list:list=None, 
                         reduction_method_list:list=None):
    if clf_list==None:
        clf_list = [key for key in clfs]
    if reduction_method_list==None:
        reduction_method_list = [key for key in reduction_methods]
    tuned_models = []
    for clf_ident in clf_list:
        for reduction_method_ident in reduction_method_list:
            tuned_models.append(tune_model(data_matrix, 
                                           data_labels, 
                                           clf_ident, 
                                           reduction_method_ident) )
    return tuned_models

def tune_model(data_matrix:np.ndarray, 
               data_labels:np.ndarray,
               clf_ident:str, 
               reduction_method_ident:str):
    #Let user know which model is being tuned
    print('Hyperparameter tuning:', clf_ident, 'using dimensionality reducuction:', reduction_method_ident)
    #obtain classifier, hyperparameters from corresponding dictionaries
    clf = clfs[clf_ident]
    hyperparam_dict = params[clf_ident]
    reduction_method = reduction_methods[reduction_method_ident]
    #apply dimensionality reduction to data and split into different experiment classes
    #For model hyperparameter tuning, do not separate A and B conditions
    #TODO: When we have more data, it might be better to tune separate models for A and B classes
        #At the moment there is not enough data to tune each class independently
    _, data_dict = Format(data_matrix, data_labels, reduction_method, trials_in_A=0)
    #tune model
    X = data_dict['X']
    y = data_dict['y']
    clf_mod = GridSearchCV(clf, hyperparam_dict, n_jobs=7)
    clf_mod.fit(X,y)
    print('Done.')
    return {
        'clf_type': clf_ident,
        'reduction_method': reduction_method_ident,
        'clf': clf_mod
    }




#### Tune n_components for CSP Data Reduction ###

In [8]:
n_component_candidates = [x for x in range(1,64)]
    
def tune_csp_args_cv(csp_models:list, 
                  n_component_list=list(range(1,64)), 
                  X:np.ndarray=data_cube, 
                  y:np.ndarray=data_labels,
                  n_folds=5):
    for clf_dict in csp_models:
        if clf_dict['reduction_method']!='csp':
            continue
        if 'reduction_method_args' not in clf_dict:
            clf_dict['reduction_method_args'] = {}
        params = clf_dict['clf'].best_params_
        mod = clf_dict['clf'].estimator
        mod.set_params(**params)
        best_n = None
        best_acc = -1
        for n_components in n_component_list:
            acc_list = []
            cv = KFold(shuffle=True, n_splits=n_folds)
            for train_idx, test_idx in cv.split(y):
                #fit csp transform
                 #Supress lengthy print statements from CSP
                    #WARNING: This will supress all warnings and errors. ONLY use if code has been
                    # tested and is known to not crash during the transform.
                with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
                    csp_transform(X[:,:,train_idx], y[train_idx], n_components)
                #Transform data
                #Supress lengthy print statements from CSP
                #WARNING: This will supress all warnings and errors ONLY use if code has been
                    # tested and is known to not crash during the transform.
                with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
                    _, data_dict = Format(X, y, csp_transform, trials_in_A=0)
                X_formatted, y_formatted = data_dict['X'], data_dict['y']
                X_tr, y_tr = X_formatted[train_idx], y_formatted[train_idx]
                X_ts, y_ts = X_formatted[test_idx], y_formatted[test_idx]
                mod.fit(X_tr, y_tr)
                y_pred = mod.predict(X_ts)
                acc_list.append(accuracy_score(y_ts, y_pred))
            ave_acc = sum(acc_list)/len(acc_list)
            if ave_acc > best_acc:
                best_acc = ave_acc
                best_n = n_components
        clf_dict['reduction_method_args']['n_components'] = best_n
    return csp_models
        

## Model Evaluation And Selection ##

## Model Evaluation Functions ##

In [9]:

def prepare_data(X_train:np.ndarray, 
                 X_test:np.ndarray, 
                 y_train:np.ndarray, 
                 reduction_method:function, 
                 reduction_method_args:dict):
    #determine if reduction method requires fitting on training data
    #the following method determines if reduction method requires data labels, if it does
    # this implies the reduction method has to be fit to the data
    if 'y' in list(inspect.signature(reduction_method).parameters):
        #fit reduction method to the data
        reduction_method(X_train, y=y_train, **reduction_method_args)
    #do dimensionality reduction on both training and testing data
    X_tr = reduction_method(X_train, **reduction_method_args)
    X_ts = reduction_method(X_test, **reduction_method_args)
    return X_tr, X_ts
    


def evaluate_model_single_condition(clf_dict:dict, 
                                    X:np.ndarray, 
                                    y:np.ndarray, 
                                    reduction_method:function, 
                                    reduction_method_args:dict=None, 
                                    n_folds:int=5):
    #array of trial accuracies
    acc_list = []
    #obtain reduction method arguments if not provided in parameters
    if reduction_method_args is None:
        if 'reduction_method_args' not in clf_dict:
            reduction_method_args = {}
        else:
            reduction_method_args = clf_dict['reduction_method_args']
    #perform k-fold cross-validation to estimate model accuracy
    cv = KFold(shuffle=True, n_splits=n_folds)
    for train_idx, test_idx in cv.split(y):
        #split data into train-test partitions
        X_tr, X_ts = X[:,:,train_idx], X[:,:,test_idx]
        y_tr, y_ts = y[train_idx], y[test_idx]
        #do the dimensionality-reducing transformation outlined by reduction_method
        data = (X_tr, X_ts, y_tr)
        X_tr, X_ts = prepare_data(*data, reduction_method, reduction_method_args)
        #assemble and train classifier
        params = clf_dict['clf'].best_params_
        mod = clf_dict['clf'].estimator
        mod.set_params(**params)
        mod.fit(X_tr, y_tr)
        #predict and obtain accuracy score
        y_pred = mod.predict(X_ts)
        acc_list.append(accuracy_score(y_ts, y_pred))
    return np.array(acc_list)

def evaluate_model_both_conditions(clf_dict:dict, 
                                   X:np.ndarray, 
                                   y:np.ndarray, 
                                   trials_in_A:int,
                                   reduction_method:function, 
                                   reduction_method_args:dict=None, 
                                   n_folds:int=5):
    #collect parameters which do not vary between A and B conditions
    test_params = [reduction_method, reduction_method_args, n_folds]
    #obtain accuracy score on A and B separately
    A_acc = evaluate_model_single_condition(clf_dict, X[:,:,:trials_in_A], y[:trials_in_A], *test_params)
    B_acc = evaluate_model_single_condition(clf_dict, X[:,:,trials_in_A:], y[trials_in_A:], *test_params)
    return A_acc, B_acc

def evaluate_models(clfs:list, 
                    X:np.ndarray, 
                    y:np.ndarray, 
                    trials_in_A:int, 
                    reduction_methods:dict, 
                    reduction_method_arg_list:dict=None,
                    n_folds:int=5):
    #assemble test conditions, test conditions consist of a tuple (classifier_dict, reduction_method, reduction_method_args)
    if reduction_method_arg_list is None:
        reduction_method_arg_list = { method:None for method in reduction_methods }
    for clf in clfs:
        reduction_method = reduction_methods[clf['reduction_method']]
        reduction_method_args = reduction_method_arg_list[clf['reduction_method']]
        A_acc, B_acc = evaluate_model_both_conditions(clf, X, y, trials_in_A, reduction_method, reduction_method_args, n_folds)
        clf['A_accuracy'] = A_acc
        clf['B_accuracy'] = B_acc
    return clfs
                             
 

In [ ]:
with open('./bin/clfs.p','rb') as clf_pickle_file:
    tuned_models = pickle.load(clf_pickle_file)

tuned_models[1]['n_components'] = 6
tuned_models[3]['n_components'] = 6

dimensionality_reduction_methods = {'lsp': calc_lrp, 'csp': csp_transform}
evaluate_models(tuned_models, data_cube, data_labels, 100, dimensionality_reduction_methods)

## End Function Definitions. Begin Model Analysis ##
### Tune Hyperparameters ###

In [9]:
%%time
# #Tune hyperparameters. WARNING: This block takes approximately 5 hours to complete on 7 cores
tuned_clfs = tune_multiple_models(data_cube, data_labels)
with open('./bin/clfs.p','wb') as clf_pickle_file:
    pickle.dump(tuned_clfs, clf_pickle_file)

Hyperparameter tuning: LDA using dimensionality reducuction: lrp


/home/dominic/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [  nan 0.715 0.715 0.69  0.57    nan]
  warnings.warn(


Done.
Hyperparameter tuning: LDA using dimensionality reducuction: csp


/home/dominic/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [  nan 0.52  0.52  0.515 0.515 0.515]
  warnings.warn(


Done.
Hyperparameter tuning: SVM using dimensionality reducuction: lrp
Done.
Hyperparameter tuning: SVM using dimensionality reducuction: csp


KeyboardInterrupt: 

### Tune n_components for CSP dimensionality reduction ###

In [10]:
%%time
#This block runs in 2 hours on 3 cores
tune_csp_args_cv(tuned_clfs, n_component_list=list(range(1,64)))
with open('./bin/clfs.p','wb') as clf_pickle_file:
    pickle.dump(tuned_clfs, clf_pickle_file)

NameError: name 'tuned_clfs' is not defined

### Evaluate All Models and Select Best Model ###

In [11]:
%%time
dimensionality_reduction_methods = {'lrp': calc_lrp, 'csp': csp_transform}
evaluate_models(tuned_clfs, data_cube, data_labels, 100, dimensionality_reduction_methods)
with open('./bin/clfs.p','wb') as clf_pickle_file:
    pickle.dump(tuned_clfs, clf_pickle_file)

NameError: name 'tuned_clfs' is not defined